### 什么是文本张量表示
将一段文本使用张量进行表示，其中一般将词汇为表示成向量，称作词向量，再由各个词向量按顺序组成矩阵形成文本表示.

In [1]:
# 举个例子
# ["人生", "该", "如何", "起头"]
# 每个词对应矩阵中的一个向量
# [[1.32, 4,32, 0,32, 5.2],
#  [3.1, 5.43, 0.34, 3.2],
#  [3.21, 5.32, 2, 4.32],
#  [2.54, 7.32, 5.12, 9.54]]

### 文本张量表示的作用
将文本表示成张量（矩阵）形式，能够使语言文本可以作为计算机处理程序的输入，进行接下来一系列的解析工作.

### 文本张量表示的方法:
- one-hot编码
- Word2vec
- Word Embedding

### 什么是one-hot词向量表示
又称独热编码，将每个词表示成具有n个元素的向量，这个词向量中只有一个元素是1，其他元素都是0，不同词汇元素为0的位置不同，其中n的大小是整个语料中不同词汇的总数

["改变", "要", "如何", "起手"]`
==>

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1]]

### onehot编码实现:

In [2]:
# 导入用于对象保存与加载的joblib
import joblib
# 导入keras中的词汇映射器Tokenizer
from keras.preprocessing.text import Tokenizer

# 假定vocab为语料集所有不同词汇集合
vocab = {"周杰伦", "陈奕迅", "王力宏", "李宗盛", "鹿晗", "吴亦凡"}
# 实例化一个词汇映射器对象
t = Tokenizer(num_words=None, char_level=False)
# 使用映射器拟合现有文本数据
t.fit_on_texts(vocab)

for token in vocab:
    zero_list = [0] * len(vocab)
    # 使用映射器转化现有文本数据, 每个词汇对应从1开始的自然数
    # 返回样式如: [[2]], 取出其中的数字需要使用[0][0]
    token_index = t.texts_to_sequences([token])[0][0] - 1
    zero_list[token_index] = 1
    print(token, "的one-hot编码为:", zero_list)

# 使用joblib工具保存映射器, 以便之后使用
tokenizer_path = "./Tokenizer"
joblib.dump(t, tokenizer_path)

李宗盛 的one-hot编码为: [1, 0, 0, 0, 0, 0]
王力宏 的one-hot编码为: [0, 1, 0, 0, 0, 0]
吴亦凡 的one-hot编码为: [0, 0, 1, 0, 0, 0]
周杰伦 的one-hot编码为: [0, 0, 0, 1, 0, 0]
陈奕迅 的one-hot编码为: [0, 0, 0, 0, 1, 0]
鹿晗 的one-hot编码为: [0, 0, 0, 0, 0, 1]


['./Tokenizer']

In [3]:
# 导入用于对象保存与加载的joblib
# from sklearn.externals import joblib
# 加载之前保存的Tokenizer, 实例化一个t对象
t = joblib.load(tokenizer_path)

# 编码token为"李宗盛"
token = "李宗盛"
# 使用t获得token_index
token_index = t.texts_to_sequences([token])[0][0] - 1
# 初始化一个zero_list
zero_list = [0] * len(vocab)
# 令zero_List的对应索引为1
zero_list[token_index] = 1
print(token, "的one-hot编码为:", zero_list)

李宗盛 的one-hot编码为: [1, 0, 0, 0, 0, 0]


### one-hot编码的优劣势：
- 优势：操作简单，容易理解.
- 劣势：完全割裂了词与词之间的联系，而且在大语料集下，每个向量的长度过大，占据大量内存

### 什么是word2vec
是一种流行的将词汇表示成向量的无监督训练方法, 该过程将构建神经网络模型, 将网络参数作为词汇的向量表示, 它包含CBOW和skipgram两种训练模式.
